In [ ]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm
import sys
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
demo = pd.read_csv('./clean_step1_demo.csv.gz',low_memory=False)
drug = pd.read_csv('./clean_step1_drug.csv.gz',low_memory=False)
indi = pd.read_csv('./clean_step1_indi.csv.gz',low_memory=False)
outc = pd.read_csv('./clean_step1_outc.csv.gz',low_memory=False)
reac = pd.read_csv('./clean_step1_reac.csv.gz',low_memory=False)
rpsr = pd.read_csv('./clean_step1_rpsr.csv.gz',low_memory=False)
ther = pd.read_csv('./clean_step1_ther.csv.gz',low_memory=False)

In [ ]:
print(demo['caseid'].nunique())
print(drug['caseid'].nunique())
print(reac['caseid'].nunique())
print(outc['caseid'].nunique())

## 1. start_dt & event_dt check

### 1-1. CDK4/6 inhibitors caseid/primaryid

In [ ]:
drug_ps = drug.query('role_cod == "PS"').reset_index(drop=True)

In [ ]:
temp = drug_ps[(drug_ps["drugname"].str.contains("ribocicli", na=False, case=False)) |
                (drug_ps["drugname"].str.contains("palbocicli", na=False, case=False)) |
                 (drug_ps["drugname"].str.contains("abemacicli", na=False, case=False)) |
                 (drug_ps["drugname"].str.contains("kisqali", na=False, case=False)) |
                 (drug_ps["drugname"].str.contains("ibrance", na=False, case=False)) |
                 (drug_ps["drugname"].str.contains("verzenio", na=False, case=False)) |
            (drug_ps["prod_ai"].str.contains("ribocicli", na=False, case=False)) |
                 (drug_ps["prod_ai"].str.contains("palbocicli", na=False, case=False)) |
            (drug_ps["prod_ai"].str.contains("abemacicli", na=False, case=False))]

In [ ]:
cdk_set = set(temp['caseid'])
print('CDK4/6 inhibitor가 primary suspect인 report의 숫자: {}'.format(len(cdk_set)))

In [ ]:
ribociclib_primaryid = set(drug_ps[(drug_ps["drugname"].str.contains("ribocicli", na=False, case=False)) |
                (drug_ps["drugname"].str.contains("kisqali", na=False, case=False)) |
                (drug_ps["prod_ai"].str.contains("ribocicli", na=False, case=False))]['primaryid'])
palbociclib_primaryid = set(drug_ps[(drug_ps["drugname"].str.contains("palbocicli", na=False, case=False)) |
                (drug_ps["drugname"].str.contains("ibrance", na=False, case=False)) |
                (drug_ps["prod_ai"].str.contains("palbocicli", na=False, case=False))]['primaryid'])
abemaciclib_primaryid = set(drug_ps[(drug_ps["drugname"].str.contains("abemacicli", na=False, case=False)) |
                (drug_ps["drugname"].str.contains("verzenio", na=False, case=False)) |
                (drug_ps["prod_ai"].str.contains("abemacicli", na=False, case=False))]['primaryid'])

### 1-2. start_dt check

In [ ]:
# Quality check of the start_dt
df_THER = ther.query('start_dt == start_dt').reset_index(drop=True)
df_THER['start_dt'] = df_THER['start_dt'].astype(int).astype(str)
df_THER['start_dt_len'] = df_THER['start_dt'].map(lambda x : len(str(x)))
df_THER = df_THER.query('start_dt_len == 8').reset_index(drop=True) # YYYYMMDD format
# Keep the first start_dt per dsg_drug_seq
df_THER = (df_THER.sort_values(by=['primaryid','caseid','dsg_drug_seq','start_dt'],ascending=True)
           .drop_duplicates(subset=['primaryid','caseid','dsg_drug_seq'],keep='first').reset_index(drop=True))

In [ ]:
df_DRUG = drug.query('caseid in @cdk_set').reset_index(drop=True)

In [ ]:
df_DRUG_THER = pd.merge(df_DRUG,df_THER[['primaryid','dsg_drug_seq','start_dt','start_dt_len']]
                       ,left_on=('primaryid','drug_seq')
                       ,right_on=('primaryid','dsg_drug_seq')
                       ,how='inner')

In [ ]:
print('The number of reports: {:,}'.format(df_DRUG_THER['primaryid'].nunique()))

In [ ]:
# Drop ilogical start_dt (<1980 or >2024)
df_DRUG_THER = df_DRUG_THER.loc[(df_DRUG_THER['start_dt'].str[:4].astype(int) >= 1980) & (df_DRUG_THER['start_dt'].str[:4].astype(int) <= 2024)].reset_index(drop=True)
df_DRUG_THER = df_DRUG_THER.loc[(df_DRUG_THER['start_dt'].str[4:6].astype(int) <= 12)].reset_index(drop=True)
print('The number of reports: {:,}'.format(df_DRUG_THER['primaryid'].nunique()))

In [ ]:
cdk_ps_primaryid_to_start_dt_dict = df_DRUG_THER.query('role_cod == "PS" and drug_seq == 1').set_index('primaryid')['start_dt'].to_dict()
pickle.dump(cdk_ps_primaryid_to_start_dt_dict,open('./cdk_ps_primaryid_to_start_dt_dict.pkl','wb'))
print('The number of reports with valid start_dt: {:,}'.format(len(cdk_ps_primaryid_to_start_dt_dict)))


In [ ]:
demo_ps_cdk = demo.query('caseid in @cdk_set').reset_index(drop=True)

In [ ]:
demo_ps_cdk = demo_ps_cdk.query('event_dt == event_dt').reset_index(drop=True)
demo_ps_cdk['event_dt'] = demo_ps_cdk['event_dt'].astype(int).astype(str)
demo_ps_cdk['event_dt_len'] = demo_ps_cdk['event_dt'].map(lambda x : len(str(x)))
demo_ps_cdk = demo_ps_cdk.query('event_dt_len == 8').reset_index(drop=True) # YYYYMMDD format

In [ ]:
cdk_ps_primaryid_to_event_dt_dict = demo_ps_cdk.set_index('primaryid')['event_dt'].to_dict()
pickle.dump(cdk_ps_primaryid_to_event_dt_dict,open('./cdk_ps_primaryid_to_event_dt_dict.pkl','wb'))
print('The number of reports with valid event_dt: {:,}'.format(len(cdk_ps_primaryid_to_event_dt_dict)))

## 2. Table 1: Characteristics

In [ ]:
demo_cdk = demo.query('caseid in @cdk_set').reset_index(drop=True)

In [ ]:
demo_cdk.loc[demo_cdk['primaryid'].isin(ribociclib_primaryid), 'drug'] = 'ribociclib'
demo_cdk.loc[demo_cdk['primaryid'].isin(palbociclib_primaryid), 'drug'] = 'palbociclib'
demo_cdk.loc[demo_cdk['primaryid'].isin(abemaciclib_primaryid), 'drug'] = 'abemaciclib'

In [ ]:
overall_counts = demo_cdk['reporter_country'].value_counts().head(4)
overall_percentages = demo_cdk['reporter_country'].value_counts(normalize=True).head(4) * 100
overall_results = pd.DataFrame({
    'Count': overall_counts,
    'Percentage': overall_percentages
})

by_drug_counts = demo_cdk.groupby('drug')['reporter_country'].value_counts().groupby(level=0).head(4)
by_drug_percentages = demo_cdk.groupby('drug')['reporter_country'].value_counts(normalize=True).groupby(level=0).head(4) * 100
by_drug_results = pd.DataFrame({
    'Count': by_drug_counts,
    'Percentage': by_drug_percentages
})

print("Top 4 Reported Countries Overall:")
print(overall_results)
print("\nTop 4 Reported Countries by Drug:")
print(by_drug_results)

In [ ]:
demo_cdk['age_yr'].describe()

In [ ]:
demo_cdk.groupby('drug')['age_yr'].describe()

In [ ]:
demo_cdk['sex'].value_counts(dropna=False)

In [ ]:
by_drug_counts = demo_cdk.groupby('drug')['sex'].value_counts().groupby(level=0).head()
by_drug_percentages = demo_cdk.groupby('drug')['sex'].value_counts(normalize=True).groupby(level=0).head()*100
by_drug_results = pd.DataFrame({'Count': by_drug_counts,'Percentage': by_drug_percentages})
by_drug_results

In [ ]:
demo_cdk['occp_cod'].value_counts(dropna=False)

In [ ]:
by_drug_counts = demo_cdk.groupby('drug')['occp_cod'].value_counts().groupby(level=0).head()
by_drug_percentages = demo_cdk.groupby('drug')['occp_cod'].value_counts(normalize=True).groupby(level=0).head()*100
by_drug_results = pd.DataFrame({'Count': by_drug_counts,'Percentage': by_drug_percentages})
by_drug_results

In [ ]:
outc.query('caseid in @cdk_set')['outc_cod'].value_counts(dropna=False)

In [ ]:
outc.query('primaryid in @abemaciclib_primaryid')['outc_cod'].value_counts()

In [ ]:
#datediff
cdk_ps_primaryid_to_start_dt_dict = pickle.load(open('./cdk_ps_primaryid_to_start_dt_dict.pkl','rb'))
cdk_ps_primaryid_to_event_dt_dict = pickle.load(open('./cdk_ps_primaryid_to_event_dt_dict.pkl','rb'))
demo_cdk['valid_start_dt'] = demo_cdk['primaryid'].map(lambda x : cdk_ps_primaryid_to_start_dt_dict.get(x,float('nan')))
demo_cdk['valid_event_dt'] = demo_cdk['primaryid'].map(lambda x : cdk_ps_primaryid_to_event_dt_dict.get(x,float('nan')))

demo_cdk_datediff = demo_cdk.query('valid_start_dt == valid_start_dt and valid_event_dt == valid_event_dt').reset_index(drop=True)
demo_cdk_datediff['datediff'] = (pd.to_datetime(demo_cdk_datediff['valid_event_dt']) - pd.to_datetime(demo_cdk_datediff['valid_start_dt'])) / np.timedelta64(1, 'D')
demo_cdk_datediff['datediff'] = demo_cdk_datediff['datediff'].astype(int)


In [ ]:
print('Total number of PS_CDK reports: {}'.format(demo_cdk.shape[0]))
print('PS_CDK reports with valid start & event date: {}'.format(demo_cdk_datediff.shape[0]))
print('PS_CDK reports with minus datediff: {}'.format(demo_cdk_datediff.query('datediff < 0').shape[0]))
print('PS_CDK reports with zero datediff: {}'.format(demo_cdk_datediff.query('datediff == 0').shape[0]))
print('PS_CDK reports with valid datediff: {}'.format(demo_cdk_datediff.query('datediff >= 0').shape[0]))

In [ ]:
demo_cdk_datediff.query('datediff >= 0 ')['datediff'].describe()

In [ ]:
demo_cdk_datediff.query('datediff >= 0 ').groupby('drug')['datediff'].describe()

In [ ]:
cdk_ps_primaryid_to_datediff_dict = demo_cdk_datediff.query('datediff >= 0').set_index('primaryid')['datediff'].to_dict()
pickle.dump(cdk_ps_primaryid_to_datediff_dict,open('./cdk_ps_primaryid_to_datediff_dict.pkl','wb'))
len(cdk_ps_primaryid_to_datediff_dict)

## 3. Adverse event

In [ ]:
pt = pd.read_csv('./meddra_27/MedAscii/pt.asc',sep='$',header=None)
pt = pt.iloc[:,:4]
pt.columns = ['pt_code','pt_name','null_field','pt_soc_code']
#pt.head(2)

In [ ]:
soc = pd.read_csv('./meddra_27/MedAscii/soc.asc',sep='$',header=None)
soc = soc.iloc[:,:2]
soc.columns = ['soc_code','soc_name']
#soc.head(2)

In [ ]:
#pt belongs to cardiac disorders
cardiac_pt_list = set(pt.query('pt_soc_code == 10007541')['pt_name'])

In [ ]:
cardiac_reac_subset = reac.query('caseid in @cdk_set and pt in @cardiac_pt_list').reset_index(drop=True)

In [ ]:
cardiac_reac_subset.drop_duplicates(subset=['caseid','pt'],inplace=True)

In [ ]:
print(cardiac_reac_subset['primaryid'].nunique())
cardiac_reac_subset['pt'].value_counts()[:5]

In [ ]:
print(cardiac_reac_subset.query('primaryid in @palbociclib_primaryid')['primaryid'].nunique())
cardiac_reac_subset.query('primaryid in @palbociclib_primaryid')['pt'].value_counts()[:5]

In [ ]:
print(cardiac_reac_subset.query('primaryid in @ribociclib_primaryid')['primaryid'].nunique())
cardiac_reac_subset.query('primaryid in @ribociclib_primaryid')['pt'].value_counts()[:5]

In [ ]:
print(cardiac_reac_subset.query('primaryid in @abemaciclib_primaryid')['primaryid'].nunique())
cardiac_reac_subset.query('primaryid in @abemaciclib_primaryid')['pt'].value_counts()[:5]

## 4. SMQ analysis

## SMQ
    20000051	Arrhythmia related investigations, signs and symptoms (SMQ)
    20000004	Cardiac failure (SMQ)
    20000150	Cardiomyopathy (SMQ)
    20000147	Hypertension (SMQ)
    20000239	Noninfectious myocarditis/pericarditis (SMQ)
    20000001	Torsade de pointes/QT prolongation (SMQ)
    20000047    Myocardial infarction (SMQ)

In [ ]:
smq_content = pd.read_csv('./meddra_27/MedAscii/smq_content.asc',sep='$',header=None)
smq_content = smq_content.iloc[:,:-1]
smq_content.columns = ['smq_code','term_code','term_level','term_scope','term_category','term_weight','term_status','term_addition_ver','term_last_ver']

In [ ]:
target_smq = 20000001
event_code = smq_content.query('smq_code == @target_smq and term_level == 4 and term_scope == 2')['term_code'] #term_level = 4 means PT ; term_scope = 2 means narrow 
PT_QT = pt.query('pt_code in @event_code')['pt_name'].to_list()

target_smq = 20000004
event_code = smq_content.query('smq_code == @target_smq and term_level == 4 and term_scope == 2')['term_code'] #term_level = 4 means PT ; term_scope = 2 means narrow 
PT_HF = pt.query('pt_code in @event_code')['pt_name'].to_list()

target_smq = 20000051
event_code = smq_content.query('smq_code == @target_smq and term_level == 4 and term_scope == 2')['term_code'] #term_level = 4 means PT ; term_scope = 2 means narrow 
PT_ARR = pt.query('pt_code in @event_code')['pt_name'].to_list()

target_smq = 20000147
event_code = smq_content.query('smq_code == @target_smq and term_level == 4 and term_scope == 2')['term_code'] #term_level = 4 means PT ; term_scope = 2 means narrow 
PT_HTN = pt.query('pt_code in @event_code')['pt_name'].to_list()

target_smq = 20000150
event_code = smq_content.query('smq_code == @target_smq and term_level == 4 and term_scope == 2')['term_code'] #term_level = 4 means PT ; term_scope = 2 means narrow 
PT_CMP = pt.query('pt_code in @event_code')['pt_name'].to_list()

target_smq = 20000239
event_code = smq_content.query('smq_code == @target_smq and term_level == 4 and term_scope == 2')['term_code'] #term_level = 4 means PT ; term_scope = 2 means narrow 
PT_CARDITIS = pt.query('pt_code in @event_code')['pt_name'].to_list()

target_smq = 20000047
event_code = smq_content.query('smq_code == @target_smq and term_level == 4 and term_scope == 2')['term_code'] #term_level = 4 means PT ; term_scope = 2 means narrow 
PT_MI = pt.query('pt_code in @event_code')['pt_name'].to_list()

In [ ]:
print(len(ribociclib_primaryid))
print(len(palbociclib_primaryid))
print(len(abemaciclib_primaryid))
print(len(ribociclib_primaryid) + len(palbociclib_primaryid) + len(abemaciclib_primaryid))

In [ ]:
drugs = {
    'ribociclib': ribociclib_primaryid,
    'palbociclib': palbociclib_primaryid,
    'abemaciclib': abemaciclib_primaryid
}
outcomes = {
    'QT': PT_QT, 'ARR':PT_ARR, 'HF':PT_HF, 'HTN':PT_HTN, 'CMP':PT_CMP, 'CARDITIS': PT_CARDITIS, 'MI': PT_MI
}
total_cases = demo['caseid'].nunique()
results = []

for drug, primaryids in drugs.items():
    for outcome, pts in outcomes.items():
        temp = reac.query('primaryid in @primaryids and pt in @pts')
        a = len(set(temp['primaryid'])) #n11
        b = len(primaryids) - a #n12
        temp = reac.query('primaryid not in @primaryids and pt in @pts') 
        c = len(set(temp['primaryid'])) #n21
        d = total_cases - a - b - c #n22
        
        results.append({
            'a': a,
            'b': b,
            'c': c,
            'd': d,
            'drug': drug,
            'outcome': outcome
        })

results_df = pd.DataFrame(results)

In [ ]:
def calculate_confidence_intervals(n11, n12, n21, n22):
    
    if n11 == 0:
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan
    prr = (n11 / (n11 + n12)) / (n21 / (n21 + n22))
    
    
    prr_se = np.sqrt((1/n11 - 1/(n11 + n12)) + (1/n21 - 1/(n21 + n22)))
    prr_lower = np.exp(np.log(prr) - 1.96 * prr_se)
    prr_upper = np.exp(np.log(prr) + 1.96 * prr_se)
    
    
    ror = (n11 / n12) / (n21 / n22)
    
    
    ror_se = np.sqrt(1/n11 + 1/n12 + 1/n21 + 1/n22)
    ror_lower = np.exp(np.log(ror) - 1.96 * ror_se)
    ror_upper = np.exp(np.log(ror) + 1.96 * ror_se)
    
    return prr, prr_lower, prr_upper, ror, ror_lower, ror_upper

def calculate_row_metrics(row, n11_col, n12_col, n21_col, n22_col, drug_col, outcome_col):
    n11 = row[n11_col]
    n12 = row[n12_col]
    n21 = row[n21_col]
    n22 = row[n22_col]
    
    n10 = n11 + n12
    n20 = n21 + n22
    n01 = n11 + n21
    n02 = n12 + n22
    n = n11 + n12 + n21 + n22
    
    PRR, PRR_lower, PRR_upper, ROR, ROR_lower, ROR_upper = calculate_confidence_intervals(n11, n12, n21, n22)
    
    e11 = n10 * n01 / n
    e12 = n10 * n02 / n
    e21 = n20 * n01 / n
    e22 = n20 * n02 / n
    
    chi_square = ((n11 - e11) ** 2 / e11 +
                  (n12 - e12) ** 2 / e12 +
                  (n21 - e21) ** 2 / e21 +
                  (n22 - e22) ** 2 / e22)
    
    IC = (np.log(2) ** -1) * (np.log((1 + n11) / (4 + n)) -
                              np.log((1 + n10) / (2 + n)) -
                              np.log((1 + n01) / (2 + n)))
    
    VIC = (np.log(2) ** -2) * ((1 + n - n10) / ((1 + n10) * (3 + n)) +
                               (1 + n - n01) / ((1 + n01) * (3 + n)) +
                               (1 + n - n11) / ((1 + n11) * (5 + n)))
    
    SIC = np.sqrt(VIC)
    IC025 = IC - 1.96 * SIC
    
    return pd.Series({
        'drug': row[drug_col],
        'outcome': row[outcome_col],
        'n11': n11,
        'n12': n12,
        'n21': n21,
        'n22': n22,
        'PRR': PRR,
        'PRR_lower': PRR_lower,
        'PRR_upper': PRR_upper,
        'ROR': ROR,
        'ROR_lower': ROR_lower,
        'ROR_upper': ROR_upper,
        'chi_square': chi_square,
        'IC': IC,
        'VIC': VIC,
        'IC025': IC025
    })

def calculate_metrics(df, n11_col, n12_col, n21_col, n22_col, drug_col, outcome_col):
    return df.progress_apply(lambda row: calculate_row_metrics(row, n11_col, n12_col, n21_col, n22_col, drug_col, outcome_col), axis=1)



In [ ]:
tqdm.pandas()
results_df_metrics = calculate_metrics(results_df, 'a', 'b', 'c', 'd', 'drug', 'outcome')

In [ ]:
results_df_metrics

In [ ]:
#results_df_metrics.to_csv('./results_df_metrics.csv',index=False)

## 5. Figure

In [ ]:
#QT_pid = set(reac.query('pt in @PT_QT')['primaryid'])
MI_pid = set(reac.query('pt in @PT_MI')['primaryid'])
HTN_pid = set(reac.query('pt in @PT_HTN')['primaryid'])
HF_pid = set(reac.query('pt in @PT_HF')['primaryid'])

In [ ]:
#demo_cdk.loc[demo_cdk['primaryid'].isin(QT_pid), 'QT'] = 1
demo_cdk.loc[demo_cdk['primaryid'].isin(MI_pid), 'MI'] = 1
demo_cdk.loc[demo_cdk['primaryid'].isin(HTN_pid), 'HTN'] = 1
demo_cdk.loc[demo_cdk['primaryid'].isin(HF_pid), 'HF'] = 1

In [ ]:
cdk_ps_primaryid_to_datediff_dict = pickle.load(open('./cdk_ps_primaryid_to_datediff_dict.pkl','rb'))
demo_cdk['datediff'] = demo_cdk['primaryid'].map(lambda x : cdk_ps_primaryid_to_datediff_dict.get(x,float('nan')))

In [ ]:
data = demo_cdk.query('datediff==datediff')[['primaryid','drug','datediff','MI','HTN','HF']]

In [ ]:
#qt_data = data[data['QT'] == 1]
mi_data = data[data['MI'] == 1]
htn_data = data[data['HTN'] == 1]
hf_data = data[data['HF'] == 1]

In [ ]:
# Create subplots
fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharey=True)
sns.set_theme(style="ticks")

# Common settings for boxplots
boxplot_kwargs = {
    'showfliers': False,
    'linewidth': 0.4,
    'palette': {'lightgrey'},
    'orient': 'h'
}

# Plot for MI
sns.boxplot(ax=axes[0], y="drug", x="datediff", data=mi_data, **boxplot_kwargs)
axes[0].set_title('Myocardial infarction', weight='bold')
axes[0].set_xlabel('Time-to-event (days)', weight='bold')
axes[0].set_ylabel('CDK4/6 inhibitors', weight='bold')

# Plot for HTN
sns.boxplot(ax=axes[1], y="drug", x="datediff", data=htn_data, **boxplot_kwargs)
axes[1].set_title('Hypertension', weight='bold')
axes[1].set_xlabel('Time-to-event (days)', weight='bold')
axes[1].set_ylabel('')

# Plot for HF
sns.boxplot(ax=axes[2], y="drug", x="datediff", data=hf_data, **boxplot_kwargs)
axes[2].set_title('Cardiac failure', weight='bold')
axes[2].set_xlabel('Time-to-event (days)', weight='bold')
axes[2].set_ylabel('')

# Adjust the appearance of the plots
for ax in axes:
    for i, box in enumerate(ax.artists):
        box.set_edgecolor('black')
        box.set_facecolor('lightgrey')
    for line in ax.lines:
        line.set_color('black')
        line.set_linewidth(1.0)
    ax.grid(False)  # Remove gridlines
    sns.despine(ax=ax, offset=0, trim=True)  # Remove spines

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()
#fig.savefig('./fig2_median_tte_mi_htn_hf.png', bbox_inches='tight', dpi=300)
#fig.savefig('./fig2_median_tte_mi_htn_hf.tiff', bbox_inches='tight', dpi=300)

In [ ]:
def calculate_statistics(data, group_by_col, value_col):
    stats = data.groupby(group_by_col)[value_col].agg(['median', 'count', lambda x: x.quantile(0.25), lambda x: x.quantile(0.75)])
    stats.columns = ['Median',  'No_of_Reports','Q1', 'Q3']
    return stats

# Calculate statistics for each drug and event
mi_stats = calculate_statistics(mi_data, 'drug', 'datediff')
htn_stats = calculate_statistics(htn_data, 'drug', 'datediff')
hf_stats = calculate_statistics(hf_data, 'drug', 'datediff')

# Combine the results into a single DataFrame
stats_df = pd.concat([mi_stats, htn_stats, hf_stats], keys=['MI', 'HTN', 'HF']).reset_index(level=1).rename_axis('Event').reset_index()
stats_df

## Supplementary table

In [ ]:
target_list = [20000001, 20000004, 20000051, 20000147, 20000150, 20000239, 20000047]
smq_selected = smq_content.query('smq_code in @target_list and term_level == 4 and term_scope == 2').reset_index(drop=True)

In [ ]:
supp_S1 = pd.merge(smq_selected[['smq_code','term_code']],pt[['pt_code','pt_name']], left_on = 'term_code', right_on = 'pt_code', how='inner')

In [ ]:
supp_S1['smq_code'].value_counts()

In [ ]:
#add smq_name and sort
#supp_S1.to_csv('./supplementary_S1.csv',index=False)